# Mapping Coordinates
As mentioned in the previous section, our aim with registration is to find a coordinate mapping between the *source* image and the *target* image. Given our discussions last week about using affine transformation matrices to covert coordinates, an obvious place to start is with registration methods that use affine methods to map between images. In this section, we will expand on the content from last week to see how we can use the matrices in the image headers to map the coordinates of one image into the coordinates of another image.

Note that registration using affine transformations is known as *linear* registration. This means we do not have free reign to align the images however we like. We cannot bend any element of the image and we cannot use different transformations in different parts of the brain. Instead, we are restricted to only being able to *translate*, *rotate*, *scale* or *shear* the images to get them to line-up. For some registration tasks, such as motion correction, this makes sense, whereas for other tasks we may need something more flexible. At the end of this lesson, we will discuss an alternative approach known as *non-linear* registration, that offers greater flexibility at a cost of greater complexity.

## Using the $\mathbf{Q}$ Matrix for Mapping
Last week, we saw how to define a matrix $\mathbf{Q}$ that can map between the voxel coordinates in an image and a transformed version of those coordinates, using a transformation matrix$\mathbf{T}$

$$
\mathbf{Q} = (\mathbf{TM})^{-1}\mathbf{M}
$$

where $\mathbf{M}$ is the voxel-to-world matrix from the image header. We also noted that this works because it turns the voxel coordinates into millimetres, applies the transformation in world-space and then converts the millimetre coordinates back into voxel coordinates. This makes sense because it is more intuitive to transform an image using real-world units, such as millimetres and degrees. Similarly, if we want to map coordinates between two different images, it makes sense to do this in world-space. This is because two images can be different dimensions, resolutions and orientations, yet we can still find the same millimetre location in each one thanks to the matrices in the header. World-space is therefore a *common coordinate system*, irrespective of the image in question.

To achieve this, we can adapt the definition of Q to be

$$
\mathbf{Q} = (\mathbf{TM}_{S})^{-1}\mathbf{M}_{T}
$$

where $\mathbf{M}_{S}$ is the header matrix from the *source* image and $\mathbf{M}_{T}$ is the header matrix from the *target* image. So what does this new version of $\mathbf{Q}$ do? If we assume that $\mathbf{x}$ represents voxel coordinates from the target image, then the operation $\mathbf{Qx}$ will first transform the voxel coordinates to millimetres using $\mathbf{M}_{T}$, then convert the millimetre coordinates to voxel coordinates in the source image using the inverse of $\mathbf{TM}_{S}$. So the important point here is that this form of $\mathbf{Q}$ will take voxel coordinates in one image and turn them into voxel coordinates in the other image. This is achieved by matched the images using the same millimetre coordinates in world-space. 

In addition, the definition of $\mathbf{Q}$ above also include a transformation $\mathbf{T}$ that can be used to adjust for any misalignment between the image. However, when we first collect these images we do not know if any transformation is necessary. As such, we have to start by considering the default mapping between the images. We can do this by setting the matrix $\mathbf{T}$ to be the identity, which is equivalent to no transformation. If we do this then $\mathbf{T}$ drops out of the equation and we have 

$$
\mathbf{Q} = (\mathbf{M}_{S})^{-1}\mathbf{M}_{T}
$$

which gives us our starting point for image registration. 

As an example, consider the following MATLAB code which loads the header matrices for the functional and anatomical images used in the previous section. The code computes $\mathbf{Q}$ using the definition above. $\mathbf{Q}$ is then used to find out which voxel in the source matches voxel $[87, 182, 127]$ from the target.

In [14]:
hdr_anat = spm_data_hdr_read('sub-01_T1w.nii');
hdr_func = spm_data_hdr_read('sub-01_task-flanker_run-1_bold.nii,1');

Mt = hdr_anat.mat;
Ms = hdr_func.mat;

Q = inv(Ms) * Mt;
Q * [87 182 127 1]'

ans = 4×1 double
    4.7972
   48.4056
   20.1250
    1.0000

According to this calculation, the matching voxel in the source image is $[4.7972, 48.4056, 20.1250]$. 

To see why, we can calculate the millimetre coordinates for voxel $[87, 182, 127]$ from the target image and the millimetre coordinates for voxel $[4.7972, 48.4056, 20.1250]$ from the source image.

In [15]:
Mt * [87     182     127    1]'
Ms * [4.7972 48.4056 20.125 1]'

ans = 4×1 double
  -83.8313
   52.0542
   -1.5000
    1.0000

ans = 4×1 double
  -83.8313
   52.0541
   -1.5000
    1.0000

So we can see that the reason these voxels are matched is because they share the same millimetre coordinates.

## Evaluating the Q matrix

We now have a way to convert coordinates between images. However, as we saw in the previous section, just because we can calculate $\mathbf{Q}$ does not mean this mapping is any good. For instance, from the example above, we have no idea if voxel $[87, 182, 127]$ and voxel $[4.7972, 48.4056, 20.1250]$ represent the same anatomical location in both images. Just because they share the same millimetre coordinates it does not mean that they correspond to the same region of the brain. This is because the starting point of world-space (the *origin*) can be defined *anywhere* in the image. In addition, the rules for changing voxels to millimetres can involve any form of affine transformation that is not necessarily the same across different images. As such, the only way to tell how well $\mathbf{Q}$ works is to visualise the images. 

In the GIF below, you can see two images in different viewing windows. When clicking on one image, the software uses the default $\mathbf{Q}$ matrix to find the closest matched voxel in the other image. As we can see, the default mapping between these images is very poor in terms of anatomical correspondence

<a title=""><img width="800" alt="Convolution of spiky function with box2" src="images/bad_reg.gif"></a>

Hopefully, our aim with image registration should now be coming into view. We know that the default mapping between two images may be poor. As such, given the definition of $\mathbf{Q}$ from above, we need to find a transformation matrix $\mathbf{T}$ that shifts the coordinates of the source image to better match the target image. Finding such a $\mathbf{T}$ matrix will be the focus of the next section.